# Getting Started With an SHFQC+, HDAWG, PQSC System

## Device Setup and Data Server

Let's start by creating a `DeviceSetup` for a control system with one SHFQC+ instrument, one HDAWG instrument, and a PQSC instrument. To learn more about what a `DeviceSetup` is and how to use it, have a look at [this page](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/00_device_setup/concepts/index.html).

First, we import `laboneq.simple`, which contains the `DeviceSetup`.

In [ ]:
from laboneq.simple import *

Create the `DeviceSetup` with the information about the data server it should connect to. Here, we use "localhost". Change this to the LabOne data serve that is running on your computer. 

Note that calling `add_dataserver` below does not attempt to make the connection yet. This will be done only upon calling `Session.connect()`.

In [ ]:
device_setup = DeviceSetup("ZI_SHFQC+_HDAWG_PQSC")
device_setup.add_dataserver(
    host="localhost",
    port="8004",
)

Create instances of the instruments, which were imported from `laboneq.simple`, and add it to the `DeviceSetup`. 

When creating the instrument instance, you need to specify the device ID under `address`; for example, `dev12345`.

If you do not have an active LabOne data server running to connect to the instrument, you also need to specify the `device_options` that are installed on your instrument. These options are used to ensure a correct experiment compilation for your system.

The possible options you can set for each instrument are: 

* SHFQC instrument:
    * either `"SHFQC/QC6CH"` if you have an 6-channel instrument, or `"SHFQC/QC4CH"` for the 4-channel version, or `"SHFQC/QC2CH"` for the 2-channel version.
    * `"PLUS"` if you have the SHFQC+;
    * `"RTR"` for the output router and adder option;
    * `"16W"` for the option to extend the avilable integration weights to 16;
  
* HDAWG instrument:
    * either `"HDAWG8"` if you have an 8-channel instrument or `"HDAWG4"` for the 4-channel version;
    * `"CNT"` for the pulse counter option;
    * `"MF"` for the multi-frequency option;
    * `"ME"` for the memory extension option;
    * `"PC"` for the real-time precompensation option;
    * `"SKW"` for the output skew control option.

* PQSC instrument: no options needed.


When passing these options to the instrument, add them in any order, separated by a forward slash (`"/"`).  Below, we will use `device_options="SHFQC/QC6CH/PLUS/16W"` for the SHFQC and `device_options="HDAWG8/MF/CNT/PC"` for the HDAWG. Have a look at [this page](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/00_device_setup/concepts/01_instrument_options.html) to learn more about the possible options of our instruments.

You can also pass additional input parameters when instantiating the instruments:
  
* the `interface` over which to connect to the instrument, either `"1GbE"` (default) or `"usb"`. Note that **to ensure the stability of the connection to the instrument, we recommend to use the ethernet interface instead of USB**;

* the `reference_clock_source` for the instrument, either as `"internal"` to use the instrument's own internal reference clock, or `"external"` (default). In the setup we have here, the SHFQC+ and HDAWG instruments should use the clock of the PQSC, so for them, we set `reference_clock_source="external"`.

In [ ]:
device_setup.add_instruments(
    SHFQC(
        uid="shfqc",
        address="dev12345",
        interface="1GbE",
        device_options="SHFQC/QC6CH/PLUS/16W",
        reference_clock_source="external",
    ),
    HDAWG(
        uid="hdawg",
        address="dev8123",
        interface="1GbE",
        device_options="HDAWG8/MF/CNT/PC",
        reference_clock_source="external",
    ),
    PQSC(uid="pqsc", address="dev10001", reference_clock_source="internal"),
)

Next, we create connections to ethe instruments and add these connections to the `DeviceSetup`. These connections are represented in LabOne Q as [logical signal lines](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/02_logical_signals/concepts/index.html) between the instruments and your device under test. Here, we assume the device under test is a six-qubit QPU and use the physical lines of these qubits as a naming convention for our signal lines. Note that the instances of the `LogicalSignal`s will be created automatically by the `add_connections` method of `DeviceSetup`, with the names that we have specified.

In [ ]:
for idx in range(6):
    device_setup.add_connections(
        "shfqc",
        create_connection(
            to_signal=f"q{idx}/drive", ports=f"SGCHANNELS/{idx}/OUTPUT", type="iq"
        ),
        create_connection(
            to_signal=f"q{idx}/measure", ports="QACHANNELS/0/OUTPUT", type="iq"
        ),
        create_connection(
            to_signal=f"q{idx}/acquire", ports="QACHANNELS/0/INPUT", type="acquire"
        ),
    )
    device_setup.add_connections(
        "hdawg",
        create_connection(to_signal=f"q{idx}/flux", ports=f"SIGOUTS/{idx}", type="rf"),
    )

You can inspect the `LogicalSignal`s that have been created by calling `device_setup.logical_signal_by_uid(signal_name)`; for example:

In [ ]:
device_setup.logical_signal_by_uid("q0/drive")

Next, we need to configure the `calibration` of the signal lines of the `DeviceSetup`. We will set the most common properties:

* `range` - the power range in dBm of the input and output ports;
* `voltage_offset` (only for the HDAWG) - the DC voltage offset to be played on each channel. Here we set 0 V for all channels;
* `local_oscillator` (only for the SHFQC+) - an instance of `Oscillator` where we specify the local oscillator frequency;
* `oscillator` - an instance of `Oscillator` where we specify the IF frequency, which, in the case of the SHFQC+, will be mixed with local oscillator frequency; and that, in the case of the HDAWG, will modulate the pulses that are played back.

Keep in mind that the channels in each of the SG channel pairs 0-1, 2-3, 4-5 share the local oscillator, so the local oscillator for these channels in these channel pairs must be configured with the same frequency value.

In [ ]:
drive_lo_01 = Oscillator(frequency=6e9)
drive_lo_23 = Oscillator(frequency=6.2e9)
drive_lo_45 = Oscillator(frequency=6.4e9)
measure_lo = Oscillator(frequency=7e9)

config = Calibration()
for idx, drive_lo in enumerate([drive_lo_01, drive_lo_23, drive_lo_45]):
    config[f"q{2 * idx}/drive"] = SignalCalibration(
        local_oscillator=drive_lo, oscillator=Oscillator(frequency=100e6), range=10
    )
    config[f"q{2 * idx + 1}/drive"] = SignalCalibration(
        local_oscillator=drive_lo, oscillator=Oscillator(frequency=100e6), range=10
    )

for idx in range(6):
    config[f"q{idx}/measure"] = SignalCalibration(
        local_oscillator=measure_lo, oscillator=Oscillator(frequency=100e6), range=5
    )
    config[f"q{idx}/acquire"] = SignalCalibration(
        local_oscillator=measure_lo, oscillator=Oscillator(frequency=100e6), range=5
    )
    config[f"q{idx}/flux"] = SignalCalibration(voltage_offset=0, range=10)

# Apply the configuration to the DeviceSetup
device_setup.set_calibration(config)

Great! We have our `DeviceSetup` for a single SHFQA+ instrument.

Before we can play a signal on the instrument, we first have to connect it to the LabOne data server via the LabOne Q [Session](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/01_session/concepts/index.html). Here, we connect in emulation mode by calling `Session.connect` with `do_emulation=True`. Set this flag to False in order to connect to a physical setup.

In [ ]:
session = Session(device_setup)
session.connect(do_emulation=True)  # do_emulation=False when at a physical setup

## Generate a Simple Experiment

Let's create a simple experiment that plays back a 50-ns gaussian pulse on all 6 SG channels of the SHFQC+ and a 100-ns constant square pulse on all 6 outputs of the HDAWG. In addition, we will sweep the amplitudes of all of these pulses. We will also play back a 2-$\mu$s gaussian-square pulse on the output port of the QA part of the SHFQC+ instrument, and run an integrated average acquisition using a constant 2-$\mu$s integration kernel.

To learn more about the `Experiment` object and how to write experiments in LabOne Q, have a look at the ["Experiment Definition"](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/05_experiment/concepts/index.html) and ["Writing and Experiment Workflow"](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/writing_experiments.html#write-the-experiment-pulse-sequence) sections of the manual.

In [ ]:
import numpy as np


@dsl.experiment(
    signals=[f"q{idx}_drive" for idx in range(6)]
    + [f"q{idx}_measure" for idx in range(6)]
    + [f"q{idx}_acquire" for idx in range(6)]
    + [f"q{idx}_flux" for idx in range(6)]
)
def simple_experiment(count):
    with dsl.acquire_loop_rt(
        count=count,
        averaging_mode=AveragingMode.CYCLIC,
        acquisition_type=AcquisitionType.INTEGRATION,
    ):
        with dsl.sweep(
            name="amplitude_sweep",
            parameter=SweepParameter("drive_pulse_amplitudes", np.linspace(0, 1, 7)),
        ) as amplitude:
            for idx in range(6):
                with dsl.section(name=f"play-drive-pulse_q{idx}"):
                    dsl.play(
                        signal=f"q{idx}_drive",
                        pulse=pulse_library.gaussian(amplitude=1, length=50e-9),
                        amplitude=amplitude,
                    )
                with dsl.section(name=f"play-flux-pulse_q{idx}") as flux_section:
                    dsl.play(
                        signal=f"q{idx}_flux",
                        pulse=pulse_library.const(amplitude=1, length=100e-9),
                        amplitude=amplitude,
                    )
                with dsl.section(
                    name=f"play-measure-pulse-and-acquire-q{idx}",
                    play_after=flux_section,
                ):
                    dsl.play(
                        signal=f"q{idx}_measure",
                        pulse=pulse_library.gaussian_square(amplitude=1, length=1e-6),
                    )
                    dsl.acquire(
                        signal=f"q{idx}_acquire",
                        handle=f"q{idx}/result",
                        kernel=pulse_library.const(amplitude=1, length=1e-6),
                    )
                    # add a processing delay
                    dsl.delay(signal=f"q{idx}_acquire", time=0.5e-6)

    # Map the ExperimentSignals "q{idx}_drive", "q{idx}_measure", "q{idx}_acquire", "q{idx}_flux", to
    # the logical signal lines defined in the `DeviceSetup`
    for idx in range(6):
        dsl.map_signal(f"q{idx}_drive", f"q{idx}/drive")
        dsl.map_signal(f"q{idx}_measure", f"q{idx}/measure")
        dsl.map_signal(f"q{idx}_acquire", f"q{idx}/acquire")
        dsl.map_signal(f"q{idx}_flux", f"q{idx}/flux")

Next, we instantiate the `Experiment` by running the function `simple_experiment`.

In [ ]:
experiment = simple_experiment(5)

Note that, the mapping between the `ExperimentSignal`s and the logical signal lines do not have to be done as part of `simple_experiment` but can also be done or modified on the `Experiment` instance returned by `simple_experiment` as follows:

```python
q0_ls = device_setup.logical_signal_groups["q0"].logical_signals
experiment.map_signal("q0_drive", q0_ls["drive"])
experiment.map_signal("q0_measure", q0_ls["measure"])
experiment.map_signal("q0_acquire", q0_ls["acquire"])
experiment.map_signal("q0_flux", q0_ls["flux"])
```

You can also map an `ExperimentSignal` directly to a `LogicalSignal` instead of its UID, as written above.

Now we compile the experiment and inspect it using `plot_simulation`:

In [ ]:
compiled_experiment = session.compile(experiment)

In [ ]:
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

plot_simulation(
    compiled_experiment,
    start_time=0,
    length=15e-6,
    signal_names_to_show=["q0_drive", "q0_flux", "q0_measure", "q0_acquire"],
)

You can inspect your experiment in more detail by using the interactive pulse-sheet viewer. Calling the function `show_pulse_sheet` creates an HTML file that you can open in your browser. To show the pulse sheet in the kernel, set `interactive=True`.

In [ ]:
show_pulse_sheet("simple_pulse_sequence", compiled_experiment, interactive=True)

Finally, let's run the experiment on the instrument. 

In [ ]:
results = session.run()

You can inspect the acquired results using the handle name that we've specified when defining the pulse sequence, `q0/result`:

In [ ]:
results["q0/result"]

Or, alternatively,

In [ ]:
results.q0.result